In [1]:
import os
import v20
import pandas as pd
import json
import requests
import logging
import time
import base64
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
from oandapyV20.endpoints.orders import OrderCreate
from oandapyV20.exceptions import V20Error
import openai
import mplfinance as mpf

# Load environment variables
load_dotenv()

# OANDA API configuration
access_token = os.getenv('OANDA_API_TOKEN')
account_id = os.getenv('OANDA_ACCOUNT_ID')
api = API(access_token=access_token, environment="practice")

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Configure logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("oandapyV20").setLevel(logging.WARNING)

# Parameters for real-time data fetching and processing
granularity = 'M5'
instrument = 'EUR_USD'
pair = 'EUR_USD'
timeframe = '5 minutes'
window_size = 576  # Adjust window size for detecting single patterns
step_size = 5      # Adjust step size accordingly

def fetch_forex_data(from_date, to_date, granularity, instrument):
    logging.info(f"Fetching forex data from {from_date} to {to_date} with granularity {granularity} for instrument {instrument}")
    params = {
        "granularity": granularity,
        "from": from_date,
        "to": to_date
    }
    data = []
    try:
        for request in InstrumentsCandlesFactory(instrument=instrument, params=params):
            response = api.request(request)
            if response:
                for candle in response.get('candles'):
                    time = candle.get('time').split('.')[0] + 'Z'
                    rec = {
                        'time': time,
                        'complete': candle['complete'],
                        'open': float(candle['mid']['o']),
                        'high': float(candle['mid']['h']),
                        'low': float(candle['mid']['l']),
                        'close': float(candle['mid']['c']),
                        'volume': candle['volume'],
                    }
                    data.append(rec)
    except Exception as e:
        logging.error(f"An error occurred fetching data: {e}")
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df

def calculate_rsi(data, length=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    fast_ema = data.ewm(span=fast_period, adjust=False).mean()
    slow_ema = data.ewm(span=slow_period, adjust=False).mean()
    macd = fast_ema - slow_ema
    signal = macd.ewm(span=signal_period, adjust=False).mean()
    return macd, signal

def calculate_bollinger_bands(data, window=20, no_of_std=2):
    rolling_mean = data.rolling(window).mean()
    rolling_std = data.rolling(window).std()
    upper_band = rolling_mean + (rolling_std * no_of_std)
    lower_band = rolling_mean - (rolling_std * no_of_std)
    return rolling_mean, upper_band, lower_band

def calculate_fibonacci_retracement(data):
    max_price = data['high'].max()
    min_price = data['low'].min()
    diff = max_price - min_price
    levels = [max_price - diff * r for r in [0.236, 0.382, 0.5, 0.618, 0.786]]
    return levels

def calculate_ichimoku(data):
    high_9 = data['high'].rolling(window=9).max()
    low_9 = data['low'].rolling(window=9).min()
    high_26 = data['high'].rolling(window=26).max()
    low_26 = data['low'].rolling(window=26).min()
    high_52 = data['high'].rolling(window=52).max()
    low_52 = data['low'].rolling(window=52).min()
    
    tenkan_sen = (high_9 + low_9) / 2
    kijun_sen = (high_26 + low_26) / 2
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)
    senkou_span_b = ((high_52 + low_52) / 2).shift(26)
    chikou_span = data['close'].shift(-26)
    
    return tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b, chikou_span

def plot_candlestick_chart(df, filename, width=600, height=400, order_details=None):
    logging.info(f"Plotting candlestick chart to {filename}")
    mc = mpf.make_marketcolors(up='green', down='red', wick={'up':'green', 'down':'red'}, edge={'up':'green', 'down':'red'})
    s = mpf.make_mpf_style(marketcolors=mc, gridstyle='--', figcolor='white', facecolor='white')
    
    addplots = []
    if 'RSI' in df.columns:
        addplots.append(mpf.make_addplot(df['RSI'], panel=1, color='blue', secondary_y=False, ylabel='RSI'))
    if 'EMA' in df.columns:
        addplots.append(mpf.make_addplot(df['EMA'], color='orange'))
    if 'MACD' in df.columns and 'MACD_Signal' in df.columns:
        addplots.append(mpf.make_addplot(df['MACD'], panel=2, color='purple', secondary_y=False, ylabel='MACD'))
        addplots.append(mpf.make_addplot(df['MACD_Signal'], panel=2, color='red'))
    if 'Bollinger_Mid' in df.columns and 'Bollinger_Upper' in df.columns and 'Bollinger_Lower' in df.columns:
        addplots.append(mpf.make_addplot(df['Bollinger_Mid'], color='blue'))
        addplots.append(mpf.make_addplot(df['Bollinger_Upper'], color='red'))
        addplots.append(mpf.make_addplot(df['Bollinger_Lower'], color='red'))
    if 'Fibonacci' in df.columns:
        for level in df['Fibonacci'].dropna().unique():
            addplots.append(mpf.make_addplot([level] * len(df), color='magenta', linestyle='--'))
    
    fig, axes = mpf.plot(df, type='candle', style=s, addplot=addplots, volume=True, figsize=(width / 100, height / 100), returnfig=True)
    
    if order_details:
        ax = axes[0]  # Main candlestick plot
        entry_price = order_details['entry_price']
        take_profit = order_details['take_profit']
        stop_loss = order_details['stop_loss']
        ax.axhline(entry_price, color='blue', linestyle='--', linewidth=2, label='Entry Price')
        ax.axhline(take_profit, color='green', linestyle='--', linewidth=2, label='Take Profit')
        ax.axhline(stop_loss, color='red', linestyle='--', linewidth=2, label='Stop Loss')
        ax.legend()
    
    fig.savefig(filename, bbox_inches='tight')
    plt.close(fig)

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_data_with_gpt4o(indicators_summary, image_base64):
    logging.info("Sending data to OpenAI API for analysis")
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {openai.api_key}"
    }
    
    prompt_content = {
        "content": "Analyze the indicators data for EUR/USD and provide order details with profit/loss greater than 2. Combine all patterns found and aggregate them to find the best pattern with the highest probability, most profit/loss, and strongest win signs.\n\n"
                   "Provide the analysis in JSON format with the following structure:\n\n"
                   "{\n"
                   "    \"orders\": [\n"
                   "        {\n"
                   "            \"timeframe\": \"# minutes\",\n"
                   "            \"pattern_name\": \"Pattern Name\",\n"
                   "            \"confidence_percentage\": ##,\n"
                   "            \"action\": \"####\",\n"
                   "            \"entry_price\": #.####,\n"
                   "            \"take_profit\": #.####,\n"
                   "            \"stop_loss\": #.####,\n"
                   "            \"profit_loss_ratio\": ##,\n"
                   "            \"deadline_date\": \"####-##-##T##:##:##Z\"\n"
                   "        }\n"
                   "    ],\n"
                   "    \"best_pattern\": {\n"
                   "        \"pattern_name\": \"Pattern Name\",\n"
                   "        \"confidence_percentage\": ##,\n"
                   "        \"action\": \"####\",\n"
                   "        \"entry_price\": #.####,\n"
                   "        \"take_profit\": #.####,\n"
                   "        \"stop_loss\": #.####,\n"
                   "        \"profit_loss_ratio\": ##,\n"
                   "        \"deadline_date\": \"####-##-##T##:##:##Z\"\n"
                   "    }\n"
                   "}\n",
        "image": f"data:image/png;base64,{image_base64}",
        "indicators_summary": indicators_summary
    }
    
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": json.dumps(prompt_content)
            }
        ],
        "max_tokens": 3000
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    try:
        response_data = response.json()
        return response_data
    except json.JSONDecodeError:
        logging.error("Failed to decode JSON response from OpenAI API")
        return None

def extract_and_place_order(response_data):
    if not response_data or "choices" not in response_data:
        logging.error("Invalid response data")
        return None

    content = response_data["choices"][0]["message"]["content"]
    start_index = content.find('{')
    end_index = content.rfind('}') + 1
    json_content = content[start_index:end_index]
    
    try:
        analysis = json.loads(json_content)
    except json.JSONDecodeError as e:
        logging.error(f"Failed to parse JSON content: {e}")
        return None
    
    orders = analysis.get("orders", [])
    best_pattern = analysis.get("best_pattern", {})
    
    for order in orders:
        if order.get("profit_loss_ratio", 0) > 2:
            logging.info(f"Order Details - Action: {order['action']}, Entry Price: {order['entry_price']}, Take Profit: {order['take_profit']}, Stop Loss: {order['stop_loss']}")
            
            order_details = {
                'action': order['action'],
                'entry_price': order['entry_price'],
                'take_profit': order['take_profit'],
                'stop_loss': order['stop_loss'],
                'deadline_date': order['deadline_date']
            }
            response = place_order(order_details)
            
            if 'orderCancelTransaction' in response:
                logging.info(f"Order {response['orderCancelTransaction']['orderID']} was canceled: {response['orderCancelTransaction']['reason']}")
            return order_details

    logging.info(f"Best Pattern Details - Action: {best_pattern['action']}, Entry Price: {best_pattern['entry_price']}, Take Profit: {best_pattern['take_profit']}, Stop Loss: {best_pattern['stop_loss']}")
    return best_pattern

def place_order(order_details):
    instrument = "EUR_USD"
    
    mkt_order = MarketOrderRequest(
        instrument=instrument,
        units=-10000 if order_details['action'].upper() == 'SELL' else 10000,
        takeProfitOnFill=TakeProfitDetails(price=order_details['take_profit']).data,
        stopLossOnFill=StopLossDetails(price=order_details['stop_loss']).data
    )
    
    r = OrderCreate(accountID=account_id, data=mkt_order.data)
    try:
        response = api.request(r)
        logging.info(f"Order placed successfully: {response}")
        return response
    except V20Error as e:
        logging.error(f"Error placing order: {e}")
        return {"error": str(e)}

def calculate_cost(data, image_size_kb):
    json_data = json.dumps(data)
    num_tokens = len(json_data.split())
    data_cost = num_tokens * 0.000005
    image_cost = image_size_kb * 0.002125
    return num_tokens, data_cost, image_cost

# Main execution
while True:
    start_time = (datetime.now(timezone.utc) - timedelta(days=2)).strftime('%Y-%m-%dT%H:%M:%SZ')
    end_time = datetime.now(timezone.utc).strftime('%Y-%m-%dT%H:%M:%SZ')

    prices = fetch_forex_data(start_time, end_time, granularity, instrument)

    prices['RSI'] = calculate_rsi(prices['close'])
    prices['EMA'] = prices['close'].ewm(span=20, adjust=False).mean()
    prices['MACD'], prices['MACD_Signal'] = calculate_macd(prices['close'])
    prices['Bollinger_Mid'], prices['Bollinger_Upper'], prices['Bollinger_Lower'] = calculate_bollinger_bands(prices['close'])
    prices['Fibonacci'] = pd.Series(calculate_fibonacci_retracement(prices))
    ichimoku = calculate_ichimoku(prices)

    # Add Ichimoku Cloud data to the DataFrame
    prices['Tenkan_Sen'], prices['Kijun_Sen'], prices['Senkou_Span_A'], prices['Senkou_Span_B'], prices['Chikou_Span'] = ichimoku

    # Collect all the data points
    indicators_summary = {
        "Prices": prices[['open', 'high', 'low', 'close', 'volume']].to_dict('records'),
        "RSI": prices['RSI'].to_list(),
        "EMA": prices['EMA'].to_list(),
        "MACD": prices['MACD'].to_list(),
        "MACD_Signal": prices['MACD_Signal'].to_list(),
        "Bollinger_Bands": {
            "Middle": prices['Bollinger_Mid'].to_list(),
            "Upper": prices['Bollinger_Upper'].to_list(),
            "Lower": prices['Bollinger_Lower'].to_list()
        },
        "Fibonacci": calculate_fibonacci_retracement(prices),
        "Ichimoku": {
            "Tenkan_Sen": prices['Tenkan_Sen'].to_list(),
            "Kijun_Sen": prices['Kijun_Sen'].to_list(),
            "Senkou_Span_A": prices['Senkou_Span_A'].to_list(),
            "Senkou_Span_B": prices['Senkou_Span_B'].to_list(),
            "Chikou_Span": prices['Chikou_Span'].to_list()
        }
    }

    num_tokens, data_cost, image_cost = calculate_cost(indicators_summary, image_size_kb=10)
    logging.info(f"Data sent: {num_tokens} tokens, Cost: ${data_cost:.6f}, Image Cost: ${image_cost:.6f}")

    # Plot and encode the image
    filename = "normal_chart.png"
    plot_candlestick_chart(prices, filename, width=700, height=500)
    image_base64 = encode_image(filename)
    image_size_kb = len(base64.b64decode(image_base64)) / 1024
    image_cost = image_size_kb * 0.002125  # Assuming $0.002125 per KB

    analysis_result = analyze_data_with_gpt4o(indicators_summary, image_base64)
    logging.info(f"OpenAI API Analysis Result: {json.dumps(analysis_result, indent=4)}")

    if analysis_result:
        response_tokens = analysis_result["usage"]["completion_tokens"]
        response_cost = response_tokens * 0.000015  # Adjust based on actual token cost
        logging.info(f"Response received: {response_tokens} tokens, Cost: ${response_cost:.6f}")

        prompt_tokens = analysis_result["usage"]["prompt_tokens"]
        prompt_cost = prompt_tokens * 0.00005  # Adjust based on actual token cost
        logging.info(f"Prompt sent: {prompt_tokens} tokens, Cost: ${prompt_cost:.6f}")



 



        order_details = extract_and_place_order(analysis_result)
    
        if order_details:
            logging.info("Order placed based on analysis.")
    
    total_cost = data_cost + response_cost + image_cost
    logging.info(f"Total cost for this run: Data: ${data_cost:.6f}, Image: ${image_cost:.6f}, Response: ${response_cost:.6f}, Total: ${total_cost:.6f}")

    logging.info("Waiting for 5 minutes before next run...")
    time.sleep(300)


INFO:root:Fetching forex data from 2024-06-01T03:55:11Z to 2024-06-03T03:55:11Z with granularity M5 for instrument EUR_USD
INFO:root:Data sent: 1891 tokens, Cost: $0.009455, Image Cost: $0.021250
INFO:root:Plotting candlestick chart to normal_chart.png
INFO:root:Sending data to OpenAI API for analysis
INFO:root:OpenAI API Analysis Result: {
    "id": "chatcmpl-9Vt4usM0OmZq3xPOrSIHFZMSjd6PW",
    "object": "chat.completion",
    "created": 1717386916,
    "model": "gpt-4o-2024-05-13",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "```json\n{\n    \"orders\": [\n        {\n            \"timeframe\": \"5 minutes\",\n            \"pattern_name\": \"RSI Oversold\",\n            \"confidence_percentage\": 95,\n            \"action\": \"Buy\",\n            \"entry_price\": 1.08512,\n            \"take_profit\": 1.0852,\n            \"stop_loss\": 1.085,\n            \"profit_loss_ratio\": 4,\n       